In [46]:
import requests
import gtfs_realtime_NYCT_pb2
import gtfs_realtime_pb2
import polars as pl
from polars import col
import re
from PIL import Image
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
from PIL import Image

In [2]:
response = requests.get(
    "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm"
)

In [3]:
feed = gtfs_realtime_pb2.FeedMessage()

In [4]:
feed.ParseFromString(response.content)

69206

In [35]:
stops = pl.read_csv(
    "stops.txt",
    separator=",",
    has_header=True,
    schema_overrides={"parent_station": str},
)

shapes = pl.read_csv(
    "shapes.txt",
    separator=",",
    has_header=True,
)

colors = pl.read_csv("MTA_Colors_20240623.csv", separator=",", has_header=True)

In [36]:
colors = colors.filter(col("Operator") == "New York City Subway")
colors = colors.with_columns(
    col("Service").str.split(",")
)  # Split the comma-delimited values into lists
colors = colors.explode("Service")  # Explode the lists into separate rows

In [37]:
shape_unpack_re = re.compile(r"^(\w{1}).*\.+(\w+)$")


def shape_unpack(shape):
    m = re.match(shape_unpack_re, shape)
    return m.group(1), m.group(2)

In [43]:
shapes_clean = shapes.with_columns(
    [
        shapes["shape_id"]
        .map_elements(lambda x: shape_unpack(x)[0], return_dtype=str)
        .alias("Line"),
        shapes["shape_id"]
        .map_elements(lambda x: shape_unpack(x)[1], return_dtype=str)
        .alias("Line_Variation"),
    ]
)

In [44]:
min(["1..N03R", "1..N13R"])

'1..N03R'

In [54]:
shapes_clean.filter(col("shape_pt_sequence") == 0)[
    ["Line", "shape_pt_lat", "shape_pt_lon"]
].write_excel("test.xlsx")

In [45]:
shapes_clean[["Line", "Line_Variation"]].group_by("Line").min().filter(col("F"))

Line,Line_Variation
str,str
"""J""","""N12R"""
"""R""","""N27R"""
"""E""","""N05R"""
"""3""","""N01R"""
"""H""","""N21R"""
…,…
"""7""","""N95R"""
"""5""","""N01R"""
"""F""","""N01R"""


In [28]:
shapes = shapes.with_columns(
    col("shape_id").map_elements(lambda x: x[0], return_dtype=str).alias("Line")
)

line_points = shapes[
    ["Line", "shape_pt_sequence", "shape_pt_lat", "shape_pt_lon"]
].unique()

In [29]:
line_points

Line,shape_pt_sequence,shape_pt_lat,shape_pt_lon
str,i64,f64,f64
"""A""",144,40.81139,-73.952117
"""S""",604,40.633373,-74.074314
"""6""",532,40.715744,-74.002925
"""R""",788,40.685024,-73.978003
"""A""",581,40.690701,-73.987738
…,…,…,…
"""A""",895,40.608717,-73.816038
"""A""",1071,40.598699,-73.762698
"""5""",361,40.664189,-73.950989


In [17]:
stop_unpack_re = re.compile(r"^(\w{1})(\d{2})")


def stop_unpack(stop):
    m = re.match(stop_unpack_re, stop)
    return m.group(1), m.group(2)

In [11]:
stops_clean = stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]].with_columns(
    [
        stops["stop_id"]
        .map_elements(lambda x: stop_unpack(x)[0], return_dtype=str)
        .alias("Line"),
        stops["stop_id"]
        .map_elements(lambda x: stop_unpack(x)[1], return_dtype=str)
        .alias("Order"),
    ]
)
stops_clean = stops_clean.join(
    line_points, left_on="Line", right_on="Service", how="left"
)

thread '<unnamed>' panicked at py-polars/src/map/series.rs:222:19:
python function failed NameError: name 'stop_unpack' is not defined
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: python function failed NameError: name 'stop_unpack' is not defined

In [16]:
stops_clean

stop_id,stop_name,stop_lat,stop_lon,Line,Order
str,str,f64,f64,str,str
"""101""","""Van Cortlandt Park-242 St""",40.889248,-73.898583,"""1""","""01"""
"""101N""","""Van Cortlandt Park-242 St""",40.889248,-73.898583,"""1""","""01"""
"""101S""","""Van Cortlandt Park-242 St""",40.889248,-73.898583,"""1""","""01"""
"""103""","""238 St""",40.884667,-73.90087,"""1""","""03"""
"""103N""","""238 St""",40.884667,-73.90087,"""1""","""03"""
…,…,…,…,…,…
"""S30N""","""Tompkinsville""",40.636949,-74.074835,"""S""","""30"""
"""S30S""","""Tompkinsville""",40.636949,-74.074835,"""S""","""30"""
"""S31""","""St George""",40.643748,-74.073643,"""S""","""31"""


In [9]:
stops_1 = stops.filter(
    col("stop_id").map_elements(
        lambda x: True if re.search(r"^1\d{2}$", x) else False, return_dtype=pl.Boolean
    )
)
stops_1_ = stops_1.sort("stop_id").to_dicts()
stops_1a = (
    stops_1.sort("stop_id")
    .select(["stop_name", "stop_lat", "stop_lon"])
    .to_dict(as_series=False)
)

In [10]:
stops_Q = stops.filter(
    col("stop_id").map_elements(
        lambda x: True if re.search(r"^Q\d{2}$", x) else False, return_dtype=pl.Boolean
    )
)
stops_Q_ = stops_Q.sort("stop_id").to_dicts()
stops_Qa = (
    stops_Q.sort("stop_id")
    .select(["stop_name", "stop_lat", "stop_lon"])
    .to_dict(as_series=False)
)

In [11]:
stops_JJ = shapes.filter(
    col("shape_id").map_elements(
        lambda x: True if re.search(r"^1\.\.N03R", x) else False,
        return_dtype=pl.Boolean,
    )
)
stops_JJ_ = stops_JJ.sort("shape_pt_sequence", descending=False).to_dicts()
stops_JJa = (
    stops_JJ.sort("shape_pt_sequence", descending=False)
    .select(["shape_pt_sequence", "shape_pt_lat", "shape_pt_lon"])
    .to_dict(as_series=False)
)

In [17]:
# Assuming stops_1a is your DataFrame
# For this example, let's recreate a similar DataFrame as your stops_1a

# Create the figure
fig = go.Figure()

# Add the scatter mapbox trace for the lines
for line in stops_clean['Line'].unique():
    d = stops_clean.filter('Line' == line).to_dict()
    fig.add_trace(
        go.Scattermapbox(
            mode="markers",
            lon=d["stop_lon"],
            lat=d["stop_lat"],
            text=d["stop_name"],
            marker={"size": 10},
            line={"width": 4, "color": "#EE352E"},
        )
    )

fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lon=stops_JJa["shape_pt_lon"],
        lat=stops_JJa["shape_pt_lat"],
        text=stops_JJa["shape_pt_sequence"],
        marker={"size": 10},
        line={"width": 4, "color": "#00933C"},
    )
)

# fig.add_trace(
#     go.Scattermapbox(
#         mode="markers",
#         lon=[-73.9851],  # Example longitude
#         lat=[40.7580],  # Example latitude
#         text=[
#             "<b>Times Square</b><br>"
#             "Location: New York<br>"
#             "Description: A major commercial intersection, "
#             "tourist destination, entertainment center, and "
#             "neighborhood in the Midtown Manhattan section of New York City."
#         ],  # Popup text with HTML
#     )
# )

# Update the layout
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center={"lat": 40.75, "lon": -73.95},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

# fig.add_layout_image(
#     dict(
#         source="https://raw.githubusercontent.com/cldougl/plot_images/add_r_img/vox.png",
#         xref="paper",
#         yref="paper",
#         x=1,
#         y=1.05,
#         sizex=0.2,
#         sizey=0.2,
#         xanchor="right",
#         yanchor="bottom",
#     )
# )

# fig.add_layout_image(dict(source="Q_train.png", x=40.75, y=-73.95))

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([dcc.Graph(id="live-map", figure=fig)])

if __name__ == "__main__":
    app.run_server(debug=True)